# Regression House Pricing Assignment

# Import Libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
os.chdir('/Users/rahulbanerjee/Rahul/ETL_HIVE/ETL_Hive_Assignments/Regression_Assignment_06_march_24')

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import matplotlib.pyplot as plt
import pickle
from pandas_profiling import ProfileReport
import numpy as np

from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.linear_model import Ridge,Lasso ,RidgeCV,LassoCV , ElasticNet , ElasticNetCV, LinearRegression

# Data Gathering/Data collection/reading csv file

In [2]:
training_df=pd.read_csv('training_set.csv')
testing_df = pd.read_csv('testing_set.csv')

In [3]:
combined_df =  pd.concat([training_df, testing_df], ignore_index=True)

# EDA

In [4]:
combined_df.iloc[:1,70:80]

,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal


In [5]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             2919 non-null   int64  
 1   MSSubClass     2919 non-null   int64  
 2   MSZoning       2915 non-null   object 
 3   LotFrontage    2433 non-null   float64
 4   LotArea        2919 non-null   int64  
 5   Street         2919 non-null   object 
 6   Alley          198 non-null    object 
 7   LotShape       2919 non-null   object 
 8   LandContour    2919 non-null   object 
 9   Utilities      2917 non-null   object 
 10  LotConfig      2919 non-null   object 
 11  LandSlope      2919 non-null   object 
 12  Neighborhood   2919 non-null   object 
 13  Condition1     2919 non-null   object 
 14  Condition2     2919 non-null   object 
 15  BldgType       2919 non-null   object 
 16  HouseStyle     2919 non-null   object 
 17  OverallQual    2919 non-null   int64  
 18  OverallC

In [6]:
combined_df.shape

(2919, 81)

In [260]:
#combined_df['SalePrice'].mode()[0]

140000.0

In [295]:
#Fixing the y labels wil NAN with mode
#combined_df['SalePrice'] = combined_df['SalePrice'].fillna(combined_df['SalePrice'].mode()[0])

In [351]:
combined_df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,140000.0
2915,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,140000.0
2916,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,140000.0
2917,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,140000.0


In [7]:
#Dropping rows where y label is NAN
combined_df.drop(combined_df[combined_df['SalePrice'].isna()].index,inplace=True)

In [8]:
combined_df.shape

(1460, 81)

In [298]:
combined_df.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,2919.000000,2919.000000,2433.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2896.000000,2918.000000,...,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000
mean,1460.000000,57.137718,69.305795,10168.114080,6.089072,5.564577,1971.312778,1984.264474,102.201312,441.423235,...,93.709832,47.486811,23.098321,2.602261,16.062350,2.251799,50.825968,6.213087,2007.792737,160467.607400
std,842.787043,42.517628,23.344905,7886.996359,1.409947,1.113131,30.291442,20.894344,179.334253,455.610826,...,126.526589,67.575493,64.244246,25.188169,56.184365,35.663946,567.402211,2.714762,1.314964,59785.743435
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,730.500000,20.000000,59.000000,7478.000000,5.000000,5.000000,1953.500000,1965.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,140000.000000
50%,1460.000000,50.000000,68.000000,9453.000000,6.000000,5.000000,1973.000000,1993.000000,0.000000,368.500000,...,0.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,140000.000000
75%,2189.500000,70.000000,80.000000,11570.000000,7.000000,6.000000,2001.000000,2004.000000,164.000000,733.000000,...,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,163000.000000
max,2919.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,1424.000000,742.000000,1012.000000,508.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000,755000.000000


In [257]:
pf = ProfileReport(combined_df)
pf.to_widgets()

In [9]:
combined_df.corr()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
Id,1.000000,0.011156,-0.010601,-0.033226,-0.028365,0.012609,-0.012713,-0.021998,-0.050298,-0.005024,...,-0.029643,-0.000477,0.002889,-0.046635,0.001330,0.057044,-0.006242,0.021172,0.000712,-0.021917
MSSubClass,0.011156,1.000000,-0.386347,-0.139781,0.032628,-0.059316,0.027850,0.040581,0.022936,-0.069836,...,-0.012579,-0.006100,-0.012037,-0.043825,-0.026030,0.008283,-0.007683,-0.013585,-0.021407,-0.084284
LotFrontage,-0.010601,-0.386347,1.000000,0.426095,0.251646,-0.059213,0.123349,0.088866,0.193458,0.233633,...,0.088521,0.151972,0.010700,0.070029,0.041383,0.206167,0.003368,0.011200,0.007450,0.351799
LotArea,-0.033226,-0.139781,0.426095,1.000000,0.105806,-0.005636,0.014228,0.013788,0.104160,0.214103,...,0.171698,0.084774,-0.018340,0.020423,0.043160,0.077672,0.038068,0.001205,-0.014261,0.263843
OverallQual,-0.028365,0.032628,0.251646,0.105806,1.000000,-0.091932,0.572323,0.550684,0.411876,0.239666,...,0.238923,0.308819,-0.113937,0.030371,0.064886,0.065166,-0.031406,0.070815,-0.027347,0.790982
OverallCond,0.012609,-0.059316,-0.059213,-0.005636,-0.091932,1.000000,-0.375983,0.073741,-0.128101,-0.046231,...,-0.003334,-0.032589,0.070356,0.025504,0.054811,-0.001985,0.068777,-0.003511,0.043950,-0.077856
YearBuilt,-0.012713,0.027850,0.123349,0.014228,0.572323,-0.375983,1.000000,0.592855,0.315707,0.249503,...,0.224880,0.188686,-0.387268,0.031355,-0.050364,0.004950,-0.034383,0.012398,-0.013618,0.522897
YearRemodAdd,-0.021998,0.040581,0.088866,0.013788,0.550684,0.073741,0.592855,1.000000,0.179618,0.128451,...,0.205726,0.226298,-0.193919,0.045286,-0.038740,0.005829,-0.010286,0.021490,0.035743,0.507101
MasVnrArea,-0.050298,0.022936,0.193458,0.104160,0.411876,-0.128101,0.315707,0.179618,1.000000,0.264736,...,0.159718,0.125703,-0.110204,0.018796,0.061466,0.011723,-0.029815,-0.005965,-0.008201,0.477493
BsmtFinSF1,-0.005024,-0.069836,0.233633,0.214103,0.239666,-0.046231,0.249503,0.128451,0.264736,1.000000,...,0.204306,0.111761,-0.102303,0.026451,0.062021,0.140491,0.003571,-0.015727,0.014359,0.386420


In [355]:
combined_df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,140000.0
2915,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,140000.0
2916,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,140000.0
2917,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,140000.0


In [10]:
#OverallQual  has very high collinearity with other columns. Also dont think this is a good example or Linear Regression
x=combined_df.drop(['Id','SalePrice','OverallQual'],axis=1)
y=combined_df['SalePrice']

In [359]:
x.shape

(2919, 78)

# Missing Value Handling

In [11]:
y.shape

(1460,)

In [302]:
y.isna().sum()

0

In [12]:
#Remove for x
for i in x.columns:
    if x[i].dtypes=='object':
        x[i]=x[i].fillna(x[i].mode()[0])
    else:
        x[i]=x[i].fillna(x[i].median())

In [13]:
x.isna().sum()

MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
Street           0
                ..
MiscVal          0
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
Length: 78, dtype: int64

# NowSplit the data set into cat + con

In [14]:
cat=x.select_dtypes(include='object')
con=x.select_dtypes(exclude='object')

In [306]:
cat

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,Ex,MnPrv,Shed,WD,Normal
1,RL,Pave,Grvl,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,Attchd,RFn,TA,TA,Y,Ex,MnPrv,Shed,WD,Normal
2,RL,Pave,Grvl,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,Ex,MnPrv,Shed,WD,Normal
3,RL,Pave,Grvl,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,Ex,MnPrv,Shed,WD,Abnorml
4,RL,Pave,Grvl,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,Attchd,RFn,TA,TA,Y,Ex,MnPrv,Shed,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,RM,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,...,Attchd,Unf,TA,TA,Y,Ex,MnPrv,Shed,WD,Normal
2915,RM,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,...,CarPort,Unf,TA,TA,Y,Ex,MnPrv,Shed,WD,Abnorml
2916,RL,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,...,Detchd,Unf,TA,TA,Y,Ex,MnPrv,Shed,WD,Abnorml
2917,RL,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,...,Attchd,Unf,TA,TA,Y,Ex,MnPrv,Shed,WD,Normal


In [307]:
con

,MSSubClass,LotFrontage,LotArea,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,60,65.0,8450,5,2003,2003,196.0,706.0,0.0,150.0,...,548.0,0,61,0,0,0,0,0,2,2008
1,20,80.0,9600,8,1976,1976,0.0,978.0,0.0,284.0,...,460.0,298,0,0,0,0,0,0,5,2007
2,60,68.0,11250,5,2001,2002,162.0,486.0,0.0,434.0,...,608.0,0,42,0,0,0,0,0,9,2008
3,70,60.0,9550,5,1915,1970,0.0,216.0,0.0,540.0,...,642.0,0,35,272,0,0,0,0,2,2006
4,60,84.0,14260,5,2000,2000,350.0,655.0,0.0,490.0,...,836.0,192,84,0,0,0,0,0,12,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,160,21.0,1936,7,1970,1970,0.0,0.0,0.0,546.0,...,0.0,0,0,0,0,0,0,0,6,2006
2915,160,21.0,1894,5,1970,1970,0.0,252.0,0.0,294.0,...,286.0,0,24,0,0,0,0,0,4,2006
2916,20,160.0,20000,7,1960,1996,0.0,1224.0,0.0,0.0,...,576.0,474,0,0,0,0,0,0,9,2006
2917,85,62.0,10441,5,1992,1992,0.0,337.0,0.0,575.0,...,0.0,80,32,0,0,0,0,700,7,2006


# Categorical Column Selection

In [15]:
#Added to find the Annova based significance or P Value
cat['SalePrice']=combined_df['SalePrice']
cat

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition,SalePrice
0,RL,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,RFn,TA,TA,Y,Gd,MnPrv,Shed,WD,Normal,208500.0
1,RL,Pave,Grvl,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,RFn,TA,TA,Y,Gd,MnPrv,Shed,WD,Normal,181500.0
2,RL,Pave,Grvl,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,RFn,TA,TA,Y,Gd,MnPrv,Shed,WD,Normal,223500.0
3,RL,Pave,Grvl,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Unf,TA,TA,Y,Gd,MnPrv,Shed,WD,Abnorml,140000.0
4,RL,Pave,Grvl,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,RFn,TA,TA,Y,Gd,MnPrv,Shed,WD,Normal,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,RL,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,...,RFn,TA,TA,Y,Gd,MnPrv,Shed,WD,Normal,175000.0
1456,RL,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,...,Unf,TA,TA,Y,Gd,MnPrv,Shed,WD,Normal,210000.0
1457,RL,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,...,RFn,TA,TA,Y,Gd,GdPrv,Shed,WD,Normal,266500.0
1458,RL,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,...,Unf,TA,TA,Y,Gd,MnPrv,Shed,WD,Normal,142125.0


In [16]:
#Using Annova for feature selection
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [17]:
#Finding the P value
imp_col=[]

def col_selector(df,target):
    for i in df.columns:                  #iterate all the column
        model=ols('target~df[i]',df).fit()
        q=anova_lm(model)
        c=q.iloc[0:1,4:5]
        p_val=c.values[0][0]
        if p_val<0.05:
            print(f'Column name:{i}, p value : {p_val}')
            imp_col.append(i)

In [18]:
col_selector(cat,cat[['SalePrice']])

Column name:MSZoning, p value : 8.817633866272648e-35
Column name:LotShape, p value : 6.447523852011766e-25
Column name:LandContour, p value : 2.742216752138124e-08
Column name:LotConfig, p value : 3.1631674736044365e-06
Column name:Neighborhood, p value : 1.5586002827679646e-225
Column name:Condition1, p value : 8.904549416141385e-08
Column name:Condition2, p value : 0.043425658360950296
Column name:BldgType, p value : 2.0567364604965262e-10
Column name:HouseStyle, p value : 3.37677653512031e-25
Column name:RoofStyle, p value : 3.653523047099411e-17
Column name:RoofMatl, p value : 7.231444779989371e-08
Column name:Exterior1st, p value : 2.5860887286387707e-43
Column name:Exterior2nd, p value : 4.8421856706985465e-43
Column name:MasVnrType, p value : 1.279703531266044e-63
Column name:ExterQual, p value : 1.439551096778953e-204
Column name:ExterCond, p value : 5.106680608672633e-07
Column name:Foundation, p value : 5.791895002231574e-91
Column name:BsmtQual, p value : 2.0781200771271595

In [19]:
len(imp_col)

39

In [20]:
#Make the dataframe out of it.
selected_cat=pd.DataFrame(cat,columns=imp_col)
selected_cat

,MSZoning,LotShape,LandContour,LotConfig,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,SaleType,SaleCondition,SalePrice
0,RL,Reg,Lvl,Inside,CollgCr,Norm,Norm,1Fam,2Story,Gable,...,Attchd,RFn,TA,TA,Y,Gd,MnPrv,WD,Normal,208500.0
1,RL,Reg,Lvl,FR2,Veenker,Feedr,Norm,1Fam,1Story,Gable,...,Attchd,RFn,TA,TA,Y,Gd,MnPrv,WD,Normal,181500.0
2,RL,IR1,Lvl,Inside,CollgCr,Norm,Norm,1Fam,2Story,Gable,...,Attchd,RFn,TA,TA,Y,Gd,MnPrv,WD,Normal,223500.0
3,RL,IR1,Lvl,Corner,Crawfor,Norm,Norm,1Fam,2Story,Gable,...,Detchd,Unf,TA,TA,Y,Gd,MnPrv,WD,Abnorml,140000.0
4,RL,IR1,Lvl,FR2,NoRidge,Norm,Norm,1Fam,2Story,Gable,...,Attchd,RFn,TA,TA,Y,Gd,MnPrv,WD,Normal,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,RL,Reg,Lvl,Inside,Gilbert,Norm,Norm,1Fam,2Story,Gable,...,Attchd,RFn,TA,TA,Y,Gd,MnPrv,WD,Normal,175000.0
1456,RL,Reg,Lvl,Inside,NWAmes,Norm,Norm,1Fam,1Story,Gable,...,Attchd,Unf,TA,TA,Y,Gd,MnPrv,WD,Normal,210000.0
1457,RL,Reg,Lvl,Inside,Crawfor,Norm,Norm,1Fam,2Story,Gable,...,Attchd,RFn,TA,TA,Y,Gd,GdPrv,WD,Normal,266500.0
1458,RL,Reg,Lvl,Inside,NAmes,Norm,Norm,1Fam,1Story,Hip,...,Attchd,Unf,TA,TA,Y,Gd,MnPrv,WD,Normal,142125.0


In [21]:
#Now drop the additional Price column we added in the earlier step.
selected_cat.drop(['SalePrice'],axis=1,inplace=True)

In [370]:
selected_cat

,MSZoning,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,BldgType,HouseStyle,RoofStyle,...,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,Fence,SaleType,SaleCondition
0,RL,Reg,Lvl,Inside,Gtl,CollgCr,Norm,1Fam,2Story,Gable,...,Typ,Gd,Attchd,RFn,TA,TA,Y,MnPrv,WD,Normal
1,RL,Reg,Lvl,FR2,Gtl,Veenker,Feedr,1Fam,1Story,Gable,...,Typ,TA,Attchd,RFn,TA,TA,Y,MnPrv,WD,Normal
2,RL,IR1,Lvl,Inside,Gtl,CollgCr,Norm,1Fam,2Story,Gable,...,Typ,TA,Attchd,RFn,TA,TA,Y,MnPrv,WD,Normal
3,RL,IR1,Lvl,Corner,Gtl,Crawfor,Norm,1Fam,2Story,Gable,...,Typ,Gd,Detchd,Unf,TA,TA,Y,MnPrv,WD,Abnorml
4,RL,IR1,Lvl,FR2,Gtl,NoRidge,Norm,1Fam,2Story,Gable,...,Typ,TA,Attchd,RFn,TA,TA,Y,MnPrv,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,RM,Reg,Lvl,Inside,Gtl,MeadowV,Norm,Twnhs,2Story,Gable,...,Typ,Gd,Attchd,Unf,TA,TA,Y,MnPrv,WD,Normal
2915,RM,Reg,Lvl,Inside,Gtl,MeadowV,Norm,TwnhsE,2Story,Gable,...,Typ,Gd,CarPort,Unf,TA,TA,Y,MnPrv,WD,Abnorml
2916,RL,Reg,Lvl,Inside,Gtl,Mitchel,Norm,1Fam,1Story,Gable,...,Typ,TA,Detchd,Unf,TA,TA,Y,MnPrv,WD,Abnorml
2917,RL,Reg,Lvl,Inside,Gtl,Mitchel,Norm,1Fam,SFoyer,Gable,...,Typ,Gd,Attchd,Unf,TA,TA,Y,MnPrv,WD,Normal


# Numerical Column Selection

In [22]:
ss=StandardScaler()

In [23]:
selected_con=pd.DataFrame(ss.fit_transform(con),columns=ss.get_feature_names_out())
selected_con.shape

(1460, 35)

In [24]:
#Feature Selection using Linear Regression
la=LinearRegression()
se=SequentialFeatureSelector(la) #Applied Lasso Here

In [25]:
filtered_con=pd.DataFrame(se.fit_transform(selected_con,y),columns=se.get_feature_names_out())

In [319]:
filtered_con

,MSSubClass,LotArea,OverallCond,YearBuilt,MasVnrArea,BsmtFinSF2,TotalBsmtSF,2ndFlrSF,GrLivArea,BsmtFullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,ScreenPorch
0,0.067331,-0.217879,-0.507284,1.046258,0.529034,-0.293025,-0.444278,1.207379,0.413547,1.087334,1.232599,0.169927,-0.207698,0.986849,-0.924311,0.306418,-0.285935
1,-0.873616,-0.072044,2.188279,0.154764,-0.567016,-0.293025,0.477158,-0.785025,-0.471891,-0.818929,-0.756321,0.169927,-0.207698,-0.287758,0.623632,0.306418,-0.285935
2,0.067331,0.137197,-0.507284,0.980221,0.338903,-0.293025,-0.299027,1.235375,0.563755,1.087334,1.232599,0.169927,-0.207698,-0.287758,0.623632,0.306418,-0.285935
3,0.302568,-0.078385,-0.507284,-1.859351,-0.567016,-0.293025,-0.671232,0.978742,0.427382,1.087334,-0.756321,0.169927,-0.207698,0.349546,0.623632,1.619830,-0.285935
4,0.067331,0.518903,-0.507284,0.947203,1.390216,-0.293025,0.211621,1.671651,1.378042,1.087334,1.232599,1.385655,-0.207698,1.624153,0.623632,1.619830,-0.285935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,2.419700,-1.043937,1.289758,-0.043346,-0.567016,-0.293025,-1.147837,0.488807,-0.807883,-0.818929,1.232599,0.169927,-0.207698,-0.925062,-0.924311,-2.320407,-0.285935
2915,2.419700,-1.049263,-0.507284,-0.043346,-0.567016,-0.293025,-1.147837,0.488807,-0.807883,-0.818929,1.232599,0.169927,-0.207698,-0.287758,-0.924311,-1.006994,-0.285935
2916,-0.873616,1.246808,1.289758,-0.373528,-0.567016,-0.293025,0.390915,-0.785025,-0.546995,1.087334,-0.756321,1.385655,-0.207698,0.349546,0.623632,0.306418,-0.285935
2917,0.655424,0.034605,-0.507284,0.683057,-0.567016,-0.293025,-0.317183,-0.785025,-1.049006,-0.818929,-0.756321,0.169927,-0.207698,-0.287758,-0.924311,-2.320407,-0.285935


In [26]:
cols = se.get_feature_names_out()
cols

array(['MSSubClass', 'LotArea', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'MasVnrArea', 'TotalBsmtSF', 'GrLivArea',
       'BsmtFullBath', 'FullBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'ScreenPorch',
       'PoolArea'], dtype=object)

In [27]:
#Make the df out of the selected columns
filtered_con=pd.DataFrame(x,columns=cols)


# Now concat back the data frames

In [28]:
x_final=filtered_con.join(selected_cat)
x_final.shape

(1460, 55)

# Now starting with imputer,standardscaling + encoding

In [29]:
cat_append=[]
con_append=[]

for i in x_final.columns:
    if x_final[i].dtypes=='object':
        cat_append.append(i)
    else:
        con_append.append(i)

In [30]:
num_pipe=Pipeline(steps=[('impute',SimpleImputer(strategy='median')),('scalar',StandardScaler())])
cat_pipe=Pipeline(steps=[('impute',SimpleImputer(strategy='most_frequent')),('encode',OneHotEncoder())])

In [31]:
column_transformer=ColumnTransformer([('cat_pipe',cat_pipe,cat_append),('num_pipe',num_pipe,con_append)])

In [32]:
transformed_df=pd.DataFrame(column_transformer.fit_transform(x_final).toarray(),columns=cat_append.append(con_append))

In [327]:
transformed_df

,0,1,2,3,4,5,6,7,8,9,...,232,233,234,235,236,237,238,239,240,241
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.207379,0.413547,1.087334,1.232599,0.169927,-0.207698,0.986849,-0.924311,0.306418,-0.285935
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-0.785025,-0.471891,-0.818929,-0.756321,0.169927,-0.207698,-0.287758,0.623632,0.306418,-0.285935
2,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.235375,0.563755,1.087334,1.232599,0.169927,-0.207698,-0.287758,0.623632,0.306418,-0.285935
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.978742,0.427382,1.087334,-0.756321,0.169927,-0.207698,0.349546,0.623632,1.619830,-0.285935
4,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.671651,1.378042,1.087334,1.232599,1.385655,-0.207698,1.624153,0.623632,1.619830,-0.285935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.488807,-0.807883,-0.818929,1.232599,0.169927,-0.207698,-0.925062,-0.924311,-2.320407,-0.285935
2915,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.488807,-0.807883,-0.818929,1.232599,0.169927,-0.207698,-0.287758,-0.924311,-1.006994,-0.285935
2916,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-0.785025,-0.546995,1.087334,-0.756321,1.385655,-0.207698,0.349546,0.623632,0.306418,-0.285935
2917,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-0.785025,-1.049006,-0.818929,-0.756321,0.169927,-0.207698,-0.287758,-0.924311,-2.320407,-0.285935


# Test Train Split the data

In [33]:
x_train,x_test,y_train,y_test=train_test_split(transformed_df,y,test_size=0.2,random_state=21)

# Model Building

In [34]:
la.fit(x_train,y_train)

LinearRegression()

# Model Evaluation

In [35]:
y_pred_train=la.predict(x_train)

mse=mean_squared_error(y_pred_train,y_train)
mae=mean_absolute_error(y_pred_train,y_train)
r2=r2_score(y_pred_train,y_train)
rmse=mse**0.5

print('*'*50)
print('MSE:',mse)
print('RMSE:',rmse)
print('MAE:',mae)
print('R2 score:',r2)
print('*'*50)

**************************************************
MSE: 407567367.4623288
RMSE: 20188.29778516081
MAE: 13944.94691780822
R2 score: 0.9304502284491052
**************************************************


# Testing Data Evaluation

In [36]:
y_pred_test=la.predict(x_test)

mse1=mean_squared_error(y_pred_test,y_test)
mae1=mean_absolute_error(y_pred_test,y_test)
r21=r2_score(y_pred_test,y_test)
rmse1=mse1**0.5

print('*'*50)
print('MSE:',mse1)
print('RMSE:',rmse1)
print('MAE:',mae1)
print('R2 score:',r21)
print('*'*50)

**************************************************
MSE: 9.145140457005256e+31
RMSE: 9563022773686810.0
MAE: 864275421777008.5
R2 score: -0.0015002605629175214
**************************************************


In [188]:
#This is overfitting

# Applying L1 or Lasso Regression¶

In [37]:
#Finding the best alpha with Cross Validation (CV)
lassocv = LassoCV(alphas=None,cv= 50 , max_iter=200000, normalize=True)
lassocv.fit(x_train,y_train)

LassoCV(cv=50, max_iter=200000, normalize=True)

In [38]:
#Finding the best alpha with Cross Validation (CV)
lassocv = LassoCV(alphas=None,cv= 50 , max_iter=200000, normalize=True)
lassocv.fit(x_train,y_train)

LassoCV(cv=50, max_iter=200000, normalize=True)

In [39]:
la=Lasso(alpha=lassocv.alpha_)

In [40]:
la.fit(x_train,y_train)

Lasso(alpha=26.997635596933517)

In [50]:
#Predict for Train Data
y_pred_train_l1=la.predict(x_train)
y_pred_train_l1

array([ 65948.06540926, 122871.05624385, 148110.18477767, ...,
       121980.62698352, 133985.80288823, 143466.09357085])

In [51]:
#Finding the same error and confidence for train data . This is for L1 Data
mse=mean_squared_error(y_train,y_pred_train_l1)
mae=mean_absolute_error(y_train,y_pred_train_l1)
r2=r2_score(y_train,y_pred_train_l1)
rmse=np.sqrt(mse)

print('MSE:',mse)
print('RMSE:',rmse)
print('MAE:',mae)
print('R2 score:',r2)

MSE: 430517020.01128036
RMSE: 20748.904067715972
MAE: 14248.445093293049
R2 score: 0.9304375180273206


In [53]:
#Predict for Train Data
y_pred_test_l1=la.predict(x_test)
y_pred_test_l1

array([183222.45046888, 198398.79329094, 228704.58410936, 210561.75009409,
        77308.96697872, 321942.41105707, 180115.19674984, 370737.88265859,
       434451.84441056, 134195.15705319, 181296.13623468, 281618.36711685,
       164478.39568078, 124405.58586082, 169068.33509846, 164035.71381014,
       220099.08728888, 102726.18638131, 177963.03285408,  61174.50127428,
       193462.66533842, 126763.97156693, 230991.56603538, 104824.57999306,
       247679.2576935 , 611241.91645035, 146352.87891653, 173194.66952907,
       111314.12260716, 134415.72333328, 272453.74953839, 137541.17375126,
       147021.75550365, 155712.89552416, 242180.41494015,  87214.4770976 ,
       328021.04021489, 146554.28738063, 184971.81100857, 202604.89804575,
       203487.26944079, 169672.51471861, 147340.20562738, 293425.85110502,
        75612.74014434, 155171.51766508, 155418.22786667, 359493.43413449,
       230470.7624993 , 233472.88739948, 102105.93542421, 196613.53740831,
        69069.97750643, 3

In [54]:
#Finding the same error and confidence for test data . This is for L1 Data
mse=mean_squared_error(y_test,y_pred_test_l1)
mae=mean_absolute_error(y_test,y_pred_test_l1)
r2=r2_score(y_test,y_pred_test_l1)
rmse=np.sqrt(mse)

print('MSE:',mse)
print('RMSE:',rmse)
print('MAE:',mae)
print('R2 score:',r2)

MSE: 1195045792.9371755
RMSE: 34569.43437398384
MAE: 17394.367618354878
R2 score: 0.8236696501622723


# Applying L2 or Lasso Regression¶

In [55]:
ridgecv = RidgeCV(alphas=np.random.uniform(0,10,50),cv = 10 , normalize=True)
ridgecv.fit(x_train,y_train)

RidgeCV(alphas=array([6.13367618, 2.82298208, 4.66799363, 1.52824243, 9.78602218,
       0.61572712, 4.42335638, 4.46941268, 8.59840244, 3.91265398,
       7.44596029, 0.69106026, 9.81640787, 9.71016697, 5.79494956,
       0.7765139 , 4.41832979, 9.86924132, 6.5859881 , 8.01202152,
       7.08739757, 2.84002683, 5.02309119, 0.39044558, 3.33684201,
       8.39486653, 9.40934716, 0.69945473, 3.65716097, 8.70391259,
       2.52166628, 9.12041825, 4.73061201, 0.49595837, 8.35742139,
       9.25914474, 2.76980253, 2.05374976, 1.62525697, 8.67624094,
       8.11697695, 1.00920769, 6.34805838, 0.16066635, 8.49433355,
       8.52825575, 5.55200257, 6.91765725, 1.17821031, 5.77775099]),
        cv=10, normalize=True)

In [57]:
ra=Ridge(alpha=ridgecv.alpha_)
ra.fit(x_train,y_train)

Ridge(alpha=0.3904455757084879)

In [58]:
#Predict for Train Data
y_pred_train_l2=ra.predict(x_train)
y_pred_train_l2

array([ 61506.63701393, 124709.96672529, 146159.12162944, ...,
       120649.07436421, 131323.27362825, 152910.69752812])

In [59]:
#Finding the same error and confidence for train data . This is for L2 Data
mse=mean_squared_error(y_train,y_pred_train_l2)
mae=mean_absolute_error(y_train,y_pred_train_l2)
r2=r2_score(y_train,y_pred_train_l2)
rmse=np.sqrt(mse)

print('MSE:',mse)
print('RMSE:',rmse)
print('MAE:',mae)
print('R2 score:',r2)

MSE: 438234822.7068312
RMSE: 20934.058916197577
MAE: 14174.195297580425
R2 score: 0.9291904836804232


In [61]:
#Predict for Test Data
y_pred_test_l2=ra.predict(x_test)
y_pred_test_l2

array([180599.06184348, 193555.92321727, 226346.4214923 , 211045.39603615,
        80250.11392373, 322852.36392189, 181167.23327853, 373341.26201478,
       430944.15455167, 133613.7715449 , 180372.85801673, 283770.10120387,
       167881.03367205, 125196.9367445 , 169797.47670116, 164758.26966374,
       221228.64675717, 111371.76462979, 185810.02729097,  77407.77062976,
       192928.02219188, 130043.10903699, 232379.00485993, 118141.54356245,
       248452.06963727, 590348.7648088 , 143255.15774572, 185966.39598231,
       110679.76534432, 135070.1060742 , 261652.86393396, 136607.07390633,
       148836.78812543, 153696.38326608, 233262.00548175, 101535.88454584,
       323883.19115256, 144658.70954571, 185286.95225609, 201836.46245293,
       204835.28957408, 177216.52961111, 146400.24775433, 297660.66067347,
        73819.96263934, 151647.15203104, 153274.80966656, 363341.38389429,
       230259.05413412, 230648.28882007, 102712.85257461, 186786.44524737,
        65338.29808015, 3

In [62]:
#Finding the same error and confidence for test data . This is for L2 Data
mse=mean_squared_error(y_test,y_pred_test_l2)
mae=mean_absolute_error(y_test,y_pred_test_l2)
r2=r2_score(y_test,y_pred_test_l2)
rmse=np.sqrt(mse)

print('MSE:',mse)
print('RMSE:',rmse)
print('MAE:',mae)
print('R2 score:',r2)

MSE: 1177921925.4136722
RMSE: 34320.867200781395
MAE: 18343.72297152258
R2 score: 0.826196296060563


# Summary : L1 or L2 is providing the best accuracy

# Elastic Net Regularisation

In [63]:
#Finding the best alpha with Cross Validation (CV)
elastic= ElasticNetCV(alphas=None, cv = 10 )
elastic.fit(x_train,y_train)

ElasticNetCV(cv=10)

In [64]:
elastic_net = ElasticNet(alpha=elastic.alpha_ , l1_ratio=elastic.l1_ratio_)
elastic_net.fit(x_train,y_train)

ElasticNet(alpha=112.72251337393733)

In [65]:
#Predict for Train Data
y_pred_train_en=elastic_net.predict(x_train)
y_pred_train_en

array([172318.26679805, 172537.3811313 , 181025.54242124, ...,
       177895.27367737, 177210.37642271, 175020.22085789])

In [66]:
#Finding the same error and confidence for train data . This is for Elastic Net Data
mse=mean_squared_error(y_train,y_pred_train_en)
mae=mean_absolute_error(y_train,y_pred_train_en)
r2=r2_score(y_train,y_pred_train_en)
rmse=np.sqrt(mse)

print('MSE:',mse)
print('RMSE:',rmse)
print('MAE:',mae)
print('R2 score:',r2)

MSE: 5398314964.668683
RMSE: 73473.2261757212
MAE: 52634.646741150216
R2 score: 0.1277460124508628


In [67]:
#Predict for Test Data
y_pred_test_en=elastic_net.predict(x_test)
y_pred_test_en

array([179920.05868498, 183698.60183238, 187741.75768543, 186683.06611282,
       172433.63588733, 189199.10375561, 184619.70934429, 191121.77279487,
       194670.98286726, 179081.17691918, 177739.48052325, 183237.64221285,
       182885.0102911 , 176804.56485898, 178785.69229175, 180962.31496454,
       188664.93500859, 174715.80598086, 183985.18407566, 169722.42725143,
       182623.44354341, 176013.56553268, 184357.76684773, 173004.72816737,
       185785.48480323, 201738.88106243, 177973.30583368, 177887.4348179 ,
       172682.38892991, 180417.04715548, 185341.91193051, 176857.18217654,
       183367.64699499, 179926.03266912, 181126.39482662, 172782.75487128,
       191896.14184675, 179846.80966268, 185184.5356933 , 184120.5716993 ,
       183288.06045006, 178135.75694733, 176730.01788353, 187039.64517439,
       172691.30894979, 180966.2427195 , 181923.71731146, 191742.2937022 ,
       185889.65926316, 186669.59633864, 175929.24942255, 182471.62371853,
       173648.95097791, 1

In [68]:
#Finding the same error and confidence for test data . This is for Elastic Net Data
mse=mean_squared_error(y_test,y_pred_test_en)
mae=mean_absolute_error(y_test,y_pred_test_en)
r2=r2_score(y_test,y_pred_test_en)
rmse=np.sqrt(mse)

print('MSE:',mse)
print('RMSE:',rmse)
print('MAE:',mae)
print('R2 score:',r2)

MSE: 5976438210.236823
RMSE: 77307.42661760785
MAE: 54598.65995076462
R2 score: 0.11816982527127085
